Trim Galore + STAR + FeatureCounts

# This is a cheat Notebook to run RNAseq analysis server

I am using it to have the steps for the RNAseq analaysis in one go and notbook as pipeline, and it can be run it form here but is running really running from the terminal


In [2]:
!pwd
#!ls

/DATA/home/l.gonzalez/Notebooks


In [1]:
import os, sys
import glob
import logging

## Run trim_galore for the trimming part paired-end, check first the QC with fastqc
Then remove the adaptors using trim galore (conda)
The CLI can be run directly 

In [1]:
#!conda init bash

In [ ]:
#Trimming step go where the fastq raw files are

# Original CLI command, to run this you must be in the same folder of the fastq raw files
#!parallel --xapply /home/l.gonzalez/programs/TrimGalore-0.6.5/trim_galore --illumina --paired --fastqc -o /DATA/share/pipelines/lgleon/organoids_rawdata/RNAseq/tesnotbook/ /DATA/share/pipelines/lgleon/organoids_rawdata/RNAseq/RAW_Data_20181128/::: *_R1_001.fastq.gz /DATA/share/pipelines/lgleon/organoids_rawdata/RNAseq/RAW_Data_20181128/::: *_R2_001.fastq.gz

#Here I tried to put the working directory in a var and give it in the Command, but this way it is not working
#!filefolder = /DATA/share/pipelines/lgleon/organoids_rawdata/RNAseq
#!parallel --xapply /home/l.gonzalez/programs/TrimGalore-0.6.5/trim_galore --illumina --paired -o ${filefolder}/tesnotbook/ ::: ${filefolder}/RAW_Data_20181128/*_R1_001.fastq.gz ::: ${filefolder}/RAW_Data_20181128/*_R2_001.fastq.gz


#this one WORKS !!!!!!
#Be aware that the path to the binary file, raw files and output folder are hard coded
# Change them when need it !!!!!!!!!!!!
Binary = "/home/l.gonzalez/programs/TrimGalore-0.6.5/trim_galore"
DIR_RAW_SAMPLES = "/DATA/share/pipelines/lgleon/NGS_protocol/RNA_Seq/rawData/fastq"
DIR_TRIM_Output = "/DATA/share/pipelines/lgleon/NGS_protocol/RNA_Seq/rawData/fastq/trimmed/"

!parallel --xapply /home/l.gonzalez/programs/TrimGalore-0.6.5/trim_galore \
--illumina --paired -o /DATA/share/pipelines/lgleon/NGS_protocol/RNA_Seq/rawData/fastq/trimmed/ \
::: /DATA/share/pipelines/lgleon/NGS_protocol/RNA_Seq/rawData/fastq/*.R1.fastq.gz ::: /DATA/share/pipelines/lgleon/NGS_protocol/RNA_Seq/rawData/fastq/*.R2.fastq.gz


In [13]:
!pwd

/DATA/home/l.gonzalez/Notebooks


### Run the aligner (STAR)
In this case is a small script to run STAR with paired-end files
Be aware of the path to the directory to change for the new one

In [25]:
# Variables to know where are the binary, references, input files and output
WORKING_DIR = "/DATA/share/pipelines/lgleon/organoids_rawdata/RNAseq"

DIR_INPUT_SAMPLES = "/DATA/share/pipelines/lgleon/organoids_rawdata/RNAseq/trimmed/"
DIR_BAM_OUTPUT = "/DATA/share/pipelines/lgleon/organoids_rawdata/RNAseq/bam_star/"

STAR_BINARY="/home/l.gonzalez/programs/STAR-2.7.5c/bin/Linux_x86_64_static/STAR"
GENE_REFERENCE="/DATA/share/pipelines/lgleon/References/Ensembl/STAR_Ref/Homo_sapiens.GRCh38.99.gtf"
STAR_INDEX="//DATA/share/pipelines/lgleon/References/Ensembl/newSTAR_idx"
USE_THREADS=16

In [ ]:
# The idea is run directly the small bash script /home/l.gonzalez/Notebooks/run_STAR_paried.sh
# what is in this scrript: DO NOT RUN IT, IT WILL NOT WORK LIKE THAT


#!bin/bash

# Working directory is hard coded, careful
# ENVIRONMENT
prj=/DATA/share/pipelines/lgleon/organoids_rawdata/RNAseq/
outp=/DATA/share/pipelines/lgleon/organoids_rawdata/RNAseq/bam_start_test/


# Make arrays named fq1 and fq2
fq1=(${prj}/trimmed/*_R1_001_val_1.f*q.gz)
fq2=(${prj}/trimmed/*_R2_001_val_2.f*q.gz)

#Command
for ((i=0;i<"${#fq1[@]}";i++)); do
  sample="${fq1[$i]%%_R*}"
  outprefix=${outp}/"${sample}"

/home/l.gonzalez/programs/STAR-2.7.5c/bin/Linux_x86_64_static/STAR \
--runMode alignReads \
--genomeDir /DATA/share/pipelines/lgleon/References/Ensembl/newSTAR_idx/ \
--runThreadN 16 \
--sjdbGTFfile /DATA/share/pipelines/lgleon/References/Ensembl/STAR_Ref/Homo_sapiens.GRCh38.99.gtf \
--readFilesIn  "${fq1[$i]}" "${fq2[$i]}" \
--outSAMtype BAM SortedByCoordinate \
--outFileNamePrefix $outprefix \
--outFilterMismatchNmax 10 \
--outFilterIntronMotifs None \
--chimSegmentMin 1 \
--sjdbScore 2 \
--quantMode GeneCounts \
--readFilesCommand zcat

done




### To modified the file here, just open the sh file and do it, easy peasy 
We are in /home/l.gonzalez/Notebooks

In [28]:
## Alignment with STAR, code is at /home/l.gonzalez/Notebooks/run_STAR_paried.sh
## Go to the folder where you wanto to save the results
## Check in the code the hard copy of the path to the project and change it 

!bash /home/l.gonzalez/Notebooks/run_STAR_paried.sh


Mar 15 14:38:29 ..... started STAR run
Mar 15 14:38:29 ..... loading genome
Mar 15 14:38:46 ..... processing annotations GTF
Mar 15 14:39:00 ..... inserting junctions into the genome indices
Mar 15 14:40:31 ..... started mapping
^C


### Run featureCounts
Again it is just a line in CLI, so go where you have the bam files and run it from there

In [11]:
## To run featureCounts is also a line that we can call frem here
## go to the folder where all the bam files are store after algnment 
## Be aware:
Binary = "/home/l.gonzalez/programs/subread-2.0.1-source/bin/featureCounts"

Output = "//DATA/share/pipelines/lgleon/NGS_protocol/RNA_Seq/Processed_Data/counts/NGS_adenomagenename_hg19.counts"
Input = "/DATA/share/pipelines/lgleon/NGS_protocol/RNA_Seq/Processed_Data/complete/Bam_colorectal_adenoma/*bam"

GFT_Ref = "/DATA/share/pipelines/lgleon/References/gencoderef/hg19/gencode.v19.annotation.gtf"


## run
#!/home/l.gonzalez/programs/subread-2.0.1-source/bin/featureCounts \
#-T 16 -t exon -g gene_id -O -a \
#/DATA/share/pipelines/lgleon/References/Ensembl/STAR_Ref/Homo_sapiens.GRCh38.99.gtf -F GTF/GFF \
#-o /DATA/share/pipelines/lgleon/organoids_rawdata/RNAseq/counts/fcounts_hg38.counts \
#/DATA/share/pipelines/lgleon/organoids_rawdata/RNAseq/bam_star/*bam

!/home/l.gonzalez/programs/subread-2.0.1-source/bin/featureCounts \
-T 16 -t gene -g gene_name -O -a \
/DATA/share/pipelines/lgleon/References/gencoderef/hg19/gencode.v19.annotation.gtf -F GTF/GFF \
-o /DATA/share/pipelines/lgleon/NGS_protocol/RNA_Seq/Processed_Data/counts/NGS_normal_genename_hg19.counts \
/DATA/share/pipelines/lgleon/NGS_protocol/RNA_Seq/Processed_Data/complete/Bam_normal_adjcentColon/*bam

#!/home/l.gonzalez/programs/subread-2.0.1-source/bin/featureCounts \
#-T 16 -t exon -g exon_id -O \
#-a /DATA/share/pipelines/lgleon/References/Ensembl/STAR_Ref/Homo_sapiens.GRCh38.99.gtf -F GTF/GFF \
#-o /DATA/share/pipelines/lgleon/organoids_rawdata/RNAseq/counts/exon_hg38.counts \
#/DATA/share/pipelines/lgleon/organoids_rawdata/RNAseq/bam_star/*bam


#Atrivute that can be use:
#An example of attributes included in your GTF annotation is:
#'gene_id "ENSG00000223972"; 
#gene_version "5"; 
#transcript_id "ENST00000456328"; 
#transcript_version "2"; 
#exon_number "1"; 
#gene_name "DDX11L1"; 
#gene_source "havana"; 
#gene_biotype "transcribed_unprocessed_pseudogene"; 
#transcript_name "DDX11L1-202"; 
#transcript_source "havana"; 
#transcript_biotype "processed_transcript"; 
#exon_id "ENSE00002234944"; 
#exon_version "1"; 
#tag "basic"; 
#transcript_support_level "1";' 





        ==========     _____ _    _ ____  _____  ______          _____  
        =====         / ____| |  | |  _ \|  __ \|  ____|   /\   |  __ \ 
          =====      | (___ | |  | | |_) | |__) | |__     /  \  | |  | |
            ====      \___ \| |  | |  _ <|  _  /|  __|   / /\ \ | |  | |
              ====    ____) | |__| | |_) | | \ \| |____ / ____ \| |__| |
        ==========   |_____/ \____/|____/|_|  \_\______/_/    \_\_____/
	  v2.0.1

//========================== featureCounts setting ===========================\\
||                                                                            ||
||             Input files : 18 BAM files                                     ||
||                           o 7046-004-303.bam                               ||
||                           o 7046-004-306.bam                               ||
||                           o 7046-004-311.bam                               ||
||                           o 7046-004-314.bam                

||    WARNING: Paired-end reads were found.                                   ||
||    Total alignments : 98339023                                             ||
||    Successfully assigned alignments : 83754198 (85.2%)                     ||
||    Running time : 0.16 minutes                                             ||
||                                                                            ||
|| Process BAM file 7046-004-347.bam...                                       ||
||    WARNING: Paired-end reads were found.                                   ||
||    Total alignments : 122070234                                            ||
||    Successfully assigned alignments : 104273031 (85.4%)                    ||
||    Running time : 0.20 minutes                                             ||
||                                                                            ||
|| Process BAM file 7046-004-349.bam...                                       ||
||    WARNING: Paired-end re

# Previous Code, do not use

In [ ]:
# add parent directory to library path so that we can load the molmedtk library
import os, sys
import glob
import logging
import subprocess
sys.path.insert(0, os.path.abspath('..')) # add parent directory to library path, to load the molmedtk library

#this is a filter to get in the log file anything with more level that debug
logging.basicConfig(filename="log-me.log", level= logging.DEBUG)

def ensure_dir(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

def extract_filename(fname):
    fn = os.path.basename( fname )
    pre, ext = os.path.splitext(fn)
    return pre

In [ ]:
#for fname in glob.glob(DIR_INPUT_SAMPLES + "/*.f*q.gz"):
#    print(fname)